# Sentiment Analysis Workflow

Tách lọc nhãn trước khi embedding: Điều này giảm thiểu thời gian và tài nguyên cần thiết để xử lý dữ liệu không liên quan.
Thêm cấu trúc rõ ràng hơn: Các bước như xử lý dữ liệu, tính embeddings, giảm chiều dữ liệu, và huấn luyện được phân tách rõ ràng.
Thêm kiểm tra đường dẫn: Đảm bảo file tồn tại trước khi tải hoặc xử lý.
Tối ưu hóa quá trình huấn luyện: Chuẩn hóa dữ liệu trước khi truyền vào mô hình.
Đảm bảo mô-đun hóa: Các bước như tiền xử lý và trích xuất embeddings được đóng gói thành hàm.

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.sql import SparkSession
from joblib import dump

In [5]:
# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Sentiment Analysis") \
    .getOrCreate()

# Paths for data and model storage
BASE_PATH = 'C:/Users/vmcch/BIGDATA/sentiment_analysis_project/'
W2V_EMBEDDINGS_PATH = "C:/Users/vmcch/BIGDATA/sentiment_analysis_project/data/processed/"  # Đường dẫn đến file .parquet chứa w2v embeddings
MODEL_PATH = os.path.join(BASE_PATH, 'model/logistic_regression_w2v_model.joblib')
SCALER_PATH = os.path.join(BASE_PATH, 'model/scaler_w2v.joblib')

In [6]:
def load_data_from_parquet(set_type, embedding_name="w2v"):
    """Load dữ liệu từ file parquet"""
    path = os.path.join(W2V_EMBEDDINGS_PATH, f"{set_type}/{embedding_name}.parquet")
    df = spark.read.parquet(path)
    return df

# Load dữ liệu train và test với embedding w2v
train_df = load_data_from_parquet("train", "w2v")
test_df = load_data_from_parquet("test", "w2v")

In [10]:
def convert_to_numpy(df):
    """Chuyển PySpark DataFrame sang numpy array để dùng với scikit-learn"""
    # Chuyển PySpark DataFrame thành Pandas DataFrame
    pandas_df = df.select("embedding", "label").toPandas()
    
    # Chuyển cột embedding (Vector) thành numpy array
    X = np.array([row["embedding"].toArray() for row in pandas_df[["embedding"]].to_numpy()])
    
    # Chuyển cột label thành numpy array
    y = pandas_df["label"].to_numpy()
    
    return X, y

# Chuyển đổi dữ liệu train và test
X_train, y_train = convert_to_numpy(train_df)
X_test, y_test = convert_to_numpy(test_df)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices